In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pickle
import itertools
from noise_filter import *

# Path to the images
img_folder_noisy = "../Data/Week3/qsd1_w3/"  # Update this path as necessary
img_folder_gt = "../Data/Week3/qsd1_w3/non_augmented/"  # Update this path as necessary
list_noisy_img = [f for f in os.listdir(img_folder_noisy) if f.endswith('.jpg')]
list_gt_img = [f for f in os.listdir(img_folder_gt) if f.endswith('.jpg')]
list_noisy_img.sort()
list_gt_img.sort()

In [2]:
# Create a list that stores 1 if the image has noise, 0 if not.
pkl_path="../Data/Week3/qsd1_w3/augmentations.pkl"
with open(pkl_path, "rb") as f:
    data = pickle.load(f)
    
noise_label=[]
for label in data:
    if label == "UnnamedImpulseNoise":
        noise_label.append(1)
    else:
        noise_label.append(0)
print(noise_label)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]


In [ ]:
# List to store noise scores calculated in the frequency space
noise_score = []

for i_noisy, i_gt, n_label in zip(list_noisy_img, list_gt_img, noise_label):
    noisy_img = cv2.imread(os.path.join(img_folder_noisy, i_noisy))
    gt_img = cv2.imread(os.path.join(img_folder_gt, i_gt))
    
    noise_score.append(fourier_noise_score(noisy_img, radius_ratio=0.75))
    #if n_label:
        #print(fourier_noise_score(noisy_img, radius_ratio=0.75))
        #analyze_noise(noisy_img, gt_img)

In [ ]:
# Now, threshold the noise_score to know which images have noise.
# For those with noise, perform the noise filter algorithm.
THRESHOLD = 40
psnr_list, ssim_list = [], []
for score, n_img_path, gt_img_path in zip(noise_score, list_noisy_img, list_gt_img):
    if score > THRESHOLD:
        noisy_img = cv2.imread(os.path.join(img_folder_noisy, n_img_path))
        gt_img = cv2.imread(os.path.join(img_folder_gt, gt_img_path))
        #print(classify_noise_type(noisy_img))
        #denoised_img=remove_noise_bilateral(noisy_img, d=7, sigmaColor=200, sigmaSpace=25)
        denoised_img=remove_noise_median(noisy_img, ksize=3)
        denoised_img=remove_noise_nlmeans(denoised_img, h=5, templateWindowSize=3, searchWindowSize=21)
        # Calculate PSNR and SSIM
        p, ssim_val = evaluate_denoising(denoised_img, gt_img)
        print(p, ssim_val)
        psnr_list.append(p)
        ssim_list.append(ssim_val)
        #cv2.imshow("",denoised_img)
        #cv2.waitKey(0)
#mean_psnr = np.mean(psnr_list)
#mean_ssim = np.mean(ssim_list)
#print(mean_psnr, mean_ssim)

31.22702500035015 0.8703094907597378
23.64078981006695 0.6404528693946588
33.109743102060214 0.8328106899898815
38.66744267232924 0.9352318728057526
30.658731388209027 0.8421660015269268
31.536588632427666 0.7829116723646307
34.711728325644984 0.9006845627411249


In [ ]:
# Define el grid de parámetros
h_values = [3, 5]
template_sizes = [3, 5, 7]
search_sizes = [15, 21]

"""d_values = [5, 9, 11]
sigmaColor_values = [25, 50, 75, 100]
sigmaSpace_values = [25, 50, 75, 100]"""

ksize=[3]

THRESHOLD = 40

# Prepara almacenamiento de resultados
results = []

# Genera todas las combinaciones posibles
param_grid = list(itertools.product(h_values, template_sizes, search_sizes, ksize))
#param_grid = list(itertools.product(d_values, sigmaColor_values, sigmaSpace_values, ksize))

for h, t, s, k in param_grid:
    psnr_list, ssim_list = [], []

    for score, n_img_path, gt_img_path in zip(noise_score, list_noisy_img, list_gt_img):
        if score > THRESHOLD:
            noisy_img = cv2.imread(os.path.join(img_folder_noisy, n_img_path))
            gt_img = cv2.imread(os.path.join(img_folder_gt, gt_img_path))

            # Filtra con los parámetros actuales
            denoised_img = remove_noise_median(noisy_img, ksize=k)
            denoised_img = remove_noise_nlmeans(denoised_img, h=h, templateWindowSize=t, searchWindowSize=s)
            #denoised_img = remove_noise_bilateral(denoised_img, d=h, sigmaColor=t, sigmaSpace=s)
            
            
            # Evalúa con PSNR y SSIM
            p, ssim_val = evaluate_denoising(denoised_img, gt_img)
            psnr_list.append(p)
            ssim_list.append(ssim_val)

    # Calcula media en todo el dataset
    mean_psnr = np.mean(psnr_list)
    mean_ssim = np.mean(ssim_list)

    results.append({
        'h': h,
        'template': t,
        'search': s,
        'ksize': k,
        'mean_psnr': mean_psnr,
        'mean_ssim': mean_ssim
    })

# Ordenar por PSNR o SSIM según tu preferencia
results_sorted = sorted(results, key=lambda x: (x['mean_psnr'], x['mean_ssim']), reverse=True)
best = results_sorted[0]

print("\n🔍 Resultados del grid search:")
for r in results_sorted:
    print(f"h={r['h']}, template={r['template']}, search={r['search']}, ksize={r['ksize']} → "
          f"PSNR={r['mean_psnr']:.2f}, SSIM={r['mean_ssim']:.3f}")

print("\n✅ Mejor conjunto de parámetros:")
print(f"h={best['h']}, template={best['template']}, search={best['search']}, ksize={r['ksize']}")
print(f"Media PSNR={best['mean_psnr']:.2f}, Media SSIM={best['mean_ssim']:.3f}")




🔍 Resultados del grid search:
h=5, template=3, search=21, ksize=3 → PSNR=31.32, SSIM=0.818
h=5, template=3, search=15, ksize=3 → PSNR=31.29, SSIM=0.817
h=5, template=5, search=15, ksize=3 → PSNR=31.19, SSIM=0.810
h=5, template=5, search=21, ksize=3 → PSNR=31.18, SSIM=0.809
h=3, template=7, search=21, ksize=3 → PSNR=31.17, SSIM=0.814
h=3, template=5, search=21, ksize=3 → PSNR=31.17, SSIM=0.815
h=5, template=7, search=15, ksize=3 → PSNR=31.14, SSIM=0.807
h=3, template=3, search=21, ksize=3 → PSNR=31.14, SSIM=0.815
h=5, template=7, search=21, ksize=3 → PSNR=31.11, SSIM=0.806
h=3, template=7, search=15, ksize=3 → PSNR=31.11, SSIM=0.813
h=3, template=5, search=15, ksize=3 → PSNR=31.09, SSIM=0.813
h=3, template=3, search=15, ksize=3 → PSNR=31.06, SSIM=0.813

✅ Mejor conjunto de parámetros:
h=5, template=3, search=21, ksize=3
Media PSNR=31.32, Media SSIM=0.818


In [ ]:
# noise_score ya contiene los valcommand:cellOutput.enableScrolling?b8c44762-98f0-4818-a0c9-b979ddd61aa6ores calculados por fourier_noise_score()
# noise_label debería ser 0 (sin ruido) o 1 (con ruido) para cada imagen

noise_score = np.array(noise_score)
noise_label = np.array(noise_label)  # asegúrate de que tenga la misma longitud

# 1️⃣ Visualizar las distribuciones separadas por tipo
plt.figure(figsize=(7,5))
plt.scatter(range(len(noise_score)), noise_score, c=noise_label, cmap='coolwarm', s=60)
plt.xlabel("Image index")
plt.ylabel("Fourier noise score")
plt.title("Noise score distribution (red = noisy, blue = clean)")
plt.colorbar(label="Label (0=clean, 1=noisy)")
plt.grid(True)
plt.show()

# 2️⃣ Opcional: histograma por grupo
plt.figure(figsize=(7,5))
plt.hist(noise_score[noise_label==0], bins=15, alpha=0.6, label="Clean")
plt.hist(noise_score[noise_label==1], bins=15, alpha=0.6, label="Noisy")
plt.xlabel("Noise score")
plt.ylabel("Number of images")
plt.title("Histogram of noise scores")
plt.legend()
plt.show()

# 3️⃣ Estimar threshold automáticamente (opcional)
# Por ejemplo, el punto medio entre las medias de ambos grupos
mean_clean = np.mean(noise_score[noise_label==0])
mean_noisy = np.mean(noise_score[noise_label==1])
threshold = (mean_clean + mean_noisy) / 2

print(f"Suggested threshold ≈ {threshold:.2f}")

# 4️⃣ Visualizar el threshold en el gráfico
plt.figure(figsize=(7,5))
plt.scatter(range(len(noise_score)), noise_score, c=noise_label, cmap='coolwarm', s=60)
plt.axhline(threshold, color='green', linestyle='--', label=f"Threshold = {threshold:.2f}")
plt.xlabel("Image index")
plt.ylabel("Noise score")
plt.title("Noise detection threshold visualization")
plt.legend()
plt.grid(True)
plt.show()
